# EXAMPLE : ADD FLAG CONDITIONAL ON DATA QUALITY RULES

Created Tue Jun 26 by Michelle.
Updated Fri Jan 4 by Michelle to reflect new demo instance.

## Scenario:
The customer table (878) contains account_number and customer identifier, which have the following data quality rules:
1. Account number should be 7 or 8 characters long
2. Customer identifier should be 7, 8 or 9 characters long

The data quality score for each of the rules above is calculated as: 
* The # of records that don't meet rule / Total # of records in dataset
Based on a benchmark of 1%, the data quality rules are *not* met if either of the rules have a data quality score >= 1%. 

## Introduction:

### This example uses:  
* Compose Query API:
- Get the latest query execution ID
- Get the latest query execution result (from the ID)
* Flags API to add a flag.
* Customer table: https://demo-sales.alationcatalog.com/table/878/
* This query that has been created and run in Compose: calculates the % of records that don't meet length requirements for customer id or account number. This is query 6777: https://demo-sales.alationcatalog.com/compose/query/7055/
* Upload Logical Metadata API to update article 

### Steps in the logic:
1. Set up the instance URL and token
2. Print SQL just for clarity for this document
3. Get latest execution id
4. Use latest execution id to get the latest execution results
5. Calculate data quality result based on results
6. Updates flag based on results
6. Update Article with summary of results

### To run this example yourself, you need to:
1. Update the *instance_url*
2. Update the *token*
3. Remove any existing flag with your username from table object https://demo-sales.alationcatalog.com/table/878/

## Example:

### 1. Set up instance parameters

In [9]:
import requests
import json
import urllib
import csv

instance_url = 'https://demo-sales.alationcatalog.com/' # Alation instance URL
token = 'fc6fc91c-c651-4c77-b2b4-da855f0d7bf7' # API token
headers = {'token' : token}

### 2. Get query text
*This is just for documentation sake so that this document includes the text of the query for completeness.*

In [10]:
query_id = 7055

api_url_sql = instance_url + '/integration/v1/query/{}/sql/'
response = requests.request("GET", api_url_sql.format(query_id), headers = headers)

print("API URL:",str(api_url_sql.format(query_id)))
print("Link to query in Alation:",'https://demo-sales.alationcatalog.com/query/7055/')
print("SQL:",response.text)

API URL: https://demo-sales.alationcatalog.com//integration/v1/query/7055/sql/
Link to query in Alation: https://demo-sales.alationcatalog.com/query/7055/
SQL: select 
	a.dq_acc_num,
	a.dq_custid,
	a.records,
	(a.dq_acc_num/a.records)*100 as dq_acc_num_perc,
	(a.dq_custid/a.records)*100 as dq_custid_perc
	from (
	SELECT SUM(CASE WHEN LENGTH (acount_number)=7 OR LENGTH(acount_number) =8 THEN 0 ELSE 1 END) AS dq_acc_num,
	   SUM(CASE WHEN LENGTH (custid)>=7 AND LENGTH (custid) <=9 THEN 0 ELSE 1 END) AS dq_custid,
	   COUNT(*) AS records
	FROM bank.customer) a


### 3. Get last execution and result
*The query is already run in Alation (currently no API to trigger a query to run). The execution result can be identified by an unique id. This step uses the API to get this ID and then subsequently uses the ID to get the actual results from that execution i.e. the data.*

Latest query execution id is stored as **execution_id**
Query results are stored in dictionary **a**.

In [11]:
api_url_execid = instance_url + '/integration/v1/query/{}/result/latest/'
response = requests.request("GET", api_url_execid.format(query_id), headers = headers)
execution_id = json.loads(response.content)['id']

print("API URL:",str(api_url_execid.format(execution_id)))
print("Last execution id for query number",query_id,"is",execution_id)

API URL: https://demo-sales.alationcatalog.com//integration/v1/query/10403/result/latest/
Last execution id for query number 7055 is 10403


In [12]:
api_url_result = instance_url + '/integration/v1/result/{}/csv/'
response = requests.request("GET", api_url_result.format(execution_id), headers = headers)
csv_reader = csv.reader(response.text.splitlines(), delimiter=',')
x = list(csv_reader)

n = 0
a = {}
while n < len(x[0]):
    key = x[0][n]
    value = x[1][n]
    a[key] = value 
    n += 1

print("API URL:",str(api_url_result.format(execution_id)))

print("Query results:")

for i in a:
    print(i,":", a[i])

API URL: https://demo-sales.alationcatalog.com//integration/v1/result/10403/csv/
Query results:
dq_acc_num : 6
dq_custid : 31
records : 1000
dq_acc_num_perc : 0.6000
dq_custid_perc : 3.1000


### 4. Set benchmark and calculate whether results meet benchmark
Calculate data quality result and store in variable **result**.
If result meets benchmark, add an endorsement, otherwise add warning with link to the article with details. 

In [13]:
benchmark = 1.0

if float(a['dq_acc_num_perc']) > benchmark or float(a['dq_custid_perc']) > benchmark:
    result = False
else:
    result = True
    
print("Passed data quality rules:",result)

Passed data quality rules: False


In [15]:
api_url_flag = instance_url + '/integration/flag/'

if result:
    data = {
        "flag_type" : "ENDORSEMENT",
        "subject": {
            "id":878,
            "otype":"table"
            }
        }    
else:
    data = {
        "flag_type" : "WARNING",
        "flag_reason": "<p>Automated update:</p> Did not pass data quality rules: account or customer id length. Benchmark:"+ str(benchmark) +"%. <p>Please refer to:<a data-oid=\"1593\" data-otype=\"article\" href=\"/article/1593/\">Data Quality: Customer table</a></p>",
        "subject": {
            "id":878,
            "otype":"table"
            }
        }
    
response = requests.post(api_url_flag, json=data, headers=headers)
print(response.text)

{"Error":"The flag already exists."}


### 5. Update Article
This step updates article with data quality result summary: https://demo-sales.alationcatalog.com/article/1593/data-quality-customer-table

In [16]:
import time
import getpass

# Formulate body text
_time = time.strftime("%a, %d %b %Y %H:%M:%S")
_user = getpass.getuser()


table_note = "<p><b>Results relate to table</b>: <a data-oid=\"878\" data-otype=\"table\" href=\"/table/878/\">bank.customer (Customer)</a></p>"
update_note = "<p><b>Rules last executed</b>: "+ _time +" by " + _user + "</p>"
dq_rules = "<p><b>Results based on calculation:</b> <a data-oid=\"7055\" data-otype=\"query\" href=\"/query/7055/\">Data quality rules - account and customer id</a></p><p></p>"
benchmark_note = "<br><p><b>Benchmark:</b> To meet data quality requirements, results were both required to be within " + str(benchmark) + "%.</p><p></p>"
result_formatted= "<table> <tr> <th>Field</th> <th>Failed Records</th> <th>Total Records</th> <th>Fail as % Total</th> </tr> <tr> <td>account_number</td> <td>" + str(a['dq_acc_num']) + "</td> <td>"+ str(a['records']) +"</td> <td>"+ str(a['dq_acc_num_perc']) +'</td> </tr> <tr> <td>custid</td> <td>'+ str(a['dq_custid']) +"</td> <td>"+ str(a['records']) +"</td> <td>"+ str(a['dq_custid_perc']) +"</td> </tr> </table> </body>"

body = table_note + update_note + dq_rules + benchmark_note + result_formatted



api_url_article = instance_url + '/api/v1/bulk_metadata/custom_fields/default/article?create_new=true&replace_values=true'

data = {
    "key": "Data Quality: Customer table", 
    "description":body
}

# Update article
response = requests.post(api_url_article, json=data, headers=headers)

# Print results
#print("Article: https://demo-sales-v.alationdata.com/article/1410/")
print("Article updated with text:")
print(body)
print("API URL:",str(api_url_article))
print("API response:")
print(response.text)


Article updated with text:
<p><b>Results relate to table</b>: <a data-oid="878" data-otype="table" href="/table/878/">bank.customer (Customer)</a></p><p><b>Rules last executed</b>: Thu, 14 Mar 2019 10:17:41 by michelle.lam</p><p><b>Results based on calculation:</b> <a data-oid="7055" data-otype="query" href="/query/7055/">Data quality rules - account and customer id</a></p><p></p><br><p><b>Benchmark:</b> To meet data quality requirements, results were both required to be within 1.0%.</p><p></p><table> <tr> <th>Field</th> <th>Failed Records</th> <th>Total Records</th> <th>Fail as % Total</th> </tr> <tr> <td>account_number</td> <td>6</td> <td>1000</td> <td>0.6000</td> </tr> <tr> <td>custid</td> <td>31</td> <td>1000</td> <td>3.1000</td> </tr> </table> </body>
API URL: https://demo-sales.alationcatalog.com//api/v1/bulk_metadata/custom_fields/default/article?create_new=true&replace_values=true
API response:
{"new_objects":0,"updated_objects":1,"number_received":1,"error_objects":[],"error":

### 6. Update table description

In [17]:
import requests
import json

_date = time.strftime("%Y-%m-%d") # Format: 2018-07-30

url = instance_url + '/api/v1/bulk_metadata/custom_fields/default/table?replace_values=true' 

data = {
    "key": "3.bank.customer", 
    "Data Quality Updated": _date,
    "Data Quality Status": "Yellow"
}

response = requests.post(url, json=data, headers=headers)
print(response.text)


{"new_objects":0,"updated_objects":1,"number_received":1,"error_objects":[],"error":""}
